In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
# import os
# spark_version = 'spark-3.3.0'
# os.environ['SPARK_VERSION'] = spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Install Postgres Driver
# !wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (77.5 kB/s)
Reading package li

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [4]:
df.count()

85981

In [5]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes equal or greater than 20
total_votes_20plus_df = df.filter(col("total_votes") >= 20)
total_votes_20plus_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [6]:
# Filter the data helpful_votes over total_votes is >=50%
helpful_votes_50plus_df = total_votes_20plus_df.filter(col("helpful_votes")/col("total_votes") >= .50)
helpful_votes_50plus_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [7]:
# Filter the data where a review was as part of the vine program
paid_vine_df = helpful_votes_50plus_df.filter(col("vine") == 'Y')
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...| 2014-01-13|
|         US|   50929595|R1RZ4JZO8DAP7H|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App

In [8]:
paid_vine_df.count()

3

In [21]:
# Filter the data where a review was not  as part of the vine program
unpaid_vine_df = helpful_votes_50plus_df.filter(col("vine") == 'N')
unpaid_vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [10]:
unpaid_vine_df.count()

3094

In [11]:
# filter total number of 5- star review piad
paid_star_rating_df = paid_vine_df.filter(col("star_rating") == 5)
paid_star_rating_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...| 2014-01-13|
|         US|   29142162| RQ94HI0WM8KIG|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App

In [12]:
paid_star_rating_df.count()

2

In [13]:
# filter total number of 5- star review unpiad
unpaid_star_rating_df = unpaid_vine_df.filter(col("star_rating") == 5)
unpaid_star_rating_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   52683195|R2WMF1IWN5ZYO4|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [14]:
unpaid_star_rating_df.count()

1704

In [46]:
# filter Percentage of 5- star reviews piad
percent_paid = (paid_star_rating_df.count()) / (paid_vine_df.count())*100
print("percentage of 5 star reviews for paid vine program was: %f" % percent_paid + "%")

percentage of 5 star reviews for paid vine program was: 66.666667%


In [47]:
# filter Percentage of 5- star reviews unpiad
percent_unpaid = unpaid_star_rating_df.count() / unpaid_vine_df.count()*100
print("percentage of 5 star reviews for unpaid vine program was: %f" % percent_unpaid + "%")


percentage of 5 star reviews for unpaid vine program was: 55.074337%
